In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

#------------------------------

config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': 56} ) #max: 1 gpu, 56 cpu
sess = tf.Session(config=config) 
keras.backend.set_session(sess)
#------------------------------
#variables
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5
#------------------------------
#Controller parameter

fit = False
read_data = False
#------------------------------
#read kaggle facial expression recognition challenge dataset (fer2013.csv)
#https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

#------------------------------
#construct CNN structure
model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_classes, activation='softmax'))
#------------------------------

#------------------------------

model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy']
)

#------------------------------
#function for drawing bar chart for emotion preditions
def emotion_analysis(emotions):
    plt.clf()
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))

    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')

    plt.show()

def emotion_analysis_api(emotions):
    emo = dict()
    objects = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
    for x,y in enumerate(objects):
        emo[y] = str(emotions[x])
    
    return emo    
    
    
#------------------------------


Using TensorFlow backend.


x size :  (1, 48, 48, 1)


C:\ProgramData\Anaconda3\lib\site-packages\keras_preprocessing\image.py:492: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


<Figure size 640x480 with 1 Axes>

In [2]:
from imutils.video import VideoStream
from imutils.video import FPS
from imutils import face_utils
import imutils
import time
import pickle
import cv2
import os
import dlib
import json
from datetime import datetime
from flask import Flask, request, jsonify
from flask_json import FlaskJSON, JsonError, json_response, as_json
import requests
from PIL import Image
import base64
from io import BytesIO

url="http://192.168.43.5:5000/detection"
fetchStream = 'http://192.168.43.226:8000/fetch'

firstDetect = True
lastDetect = False

app = Flask(__name__)
FlaskJSON(app)

#--------------------------------------

args = dict()
args['detector'] = 'face_detection_model'
args['embedding_model'] = 'openface_nn4.small2.v1.t7'
args['recognizer']= 'output\\recognizer.pickle'
args['le'] = 'output\\le.pickle'
args['confidence'] = 0.8

protoPath = os.path.sep.join([args["detector"], "deploy.prototxt"])
modelPath = os.path.sep.join([args["detector"],
                              "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)
embedder = cv2.dnn.readNetFromTorch(args["embedding_model"])
predictor = dlib.shape_predictor('Landmarks\\shape_predictor_68_face_landmarks.dat')

recognizer = pickle.loads(open(args["recognizer"], "rb").read())
le = pickle.loads(open(args["le"], "rb").read())


time.sleep(2.0)
#-----------------------------------------------------------------------
data = None
lastTime = time.time()

def readb64(base64_string):
        sbuf = BytesIO()
        sbuf.write(base64.b64decode(base64_string))
        pimg = Image.open(sbuf)
        return cv2.cvtColor(np.array(pimg), cv2.COLOR_RGB2BGR)
    
while True:
    # grab the frame from the threaded video stream
    
    r = requests.get(fetchStream)
    data = str(r.content)
    header, image = data.split(",")
    img , footer = image.split("*")
    print(footer)
    # footer1, footer2 = footer.split("*")
    
    
    frame = readb64(img)

    # resize the frame to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]

    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()
#     print(detections.shape[2])
    
#     if not detections.shape[2] > 0:
#         if not firstDetect:
#             requests.post(url, json=data)
#             firstDetect = True
    #print(detections[0, 0, :,2])
    conf = max(detections[0, 0, :,2])
    if conf < args["confidence"]:
            if not firstDetect:
                requests.post(url, json=data)
                firstDetect = True
                print("Write last detect !")
    else:              
        # loop over the detections
        for i in range(0, detections.shape[2]):
            # extract the confidence (i.e., probability) associated with
            # the prediction
            confidence = detections[0, 0, i, 2]

            # filter out weak detections
            if confidence > args["confidence"]:
#                 print(confidence)
                # compute the (x, y)-coordinates of the bounding box for
                # the face
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                l = startX
                t = startY
                r = endX
                b = endY
                # extract the face ROI
                face = frame[startY:endY, startX:endX]
                (fH, fW) = face.shape[:2]

                # ensure the face width and height are sufficiently large
                if fW < 20 or fH < 20:
                    continue

                faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                    (96, 96), (0, 0, 0), swapRB=True, crop=False)
                embedder.setInput(faceBlob)
                vec = embedder.forward()
                # perform classification to recognize the face
                preds = recognizer.predict_proba(vec)[0]
                j = np.argmax(preds)
                proba = preds[j]
                name = le.classes_[j]

    #             Get rectangle
                original_vertical_length = b-t
                t = int(t + (original_vertical_length)*0.15)
                b = int(b - (original_vertical_length)*0.05)

                margin = ((b-t) - (r-l))//2
                l = l - margin if (b-t-r+l)%2 == 0 else l - margin - 1
                r = r + margin
                rect_bb = dlib.rectangle(left=l, top=t, right=r, bottom=b)

    #             Draw landmarks
                shape = predictor(frame, rect_bb)
                shape = face_utils.shape_to_np(shape)

                # loop over the (x, y)-coordinates for the facial landmarks
                # and draw each of them
                for (i, (x, y)) in enumerate(shape):
                    cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
                    cv2.putText(frame, str(i + 1), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

                #Detect Emotion
                face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                x = face
                x = cv2.resize(x, dsize=(48, 48), interpolation=cv2.INTER_CUBIC)
                x = np.expand_dims(x, axis = 0)
                x = np.expand_dims(x, axis = 4)
                custom = model.predict(x)

                data = emotion_analysis_api(custom[0])
                data['time'] = time.ctime()
                data['age'] = '18'
                data['gender'] = "Male"
                data['publicKey'] = "fk3Oq2uqh3R/55ZjQAGWzAAvfn9jqGm6vxVaWC459KQ/+Rb+qkGAbr9SfmBIexLZI2/eVW1+1tdGQuEF/071ingNaH4gW/lSSbmFqLxeciX7T7ctxpT95210efGsp9RMB3buuMtOuagT3xSLjLkT23Wbb/sS79Fs8C5PMBb1VJ8="
                data['location'] = "The Vertical - Corporate Tower - The Orbit"

                if firstDetect:
                    requests.post(url, json=data)
                    firstDetect = False
                    print("Write first detect")
                    break


                y = startY - 10 if startY - 10 > 10 else startY + 10
                cv2.rectangle(frame, (startX, startY), (endX, endY),
                    (0, 255, 0), 2)
                cv2.putText(frame, "fk3Oq2uqh3R/55ZjQAGWzAAvfn9jqGm6vxVaWC459KQ", (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break


the vertical bangsar south cam 1-Sun Dec 16 2018 14:48:28 GMT+0800 (Malaysia Time)'


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:161: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


Write first detect
the vertical bangsar south cam 1-Sun Dec 16 2018 14:48:44 GMT+0800 (Malaysia Time)'
Write last detect !
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun 

the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:49:14 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:50:49 GMT+0800 (Malaysia Time)'
Write first detect
the vertical bangsar south cam 1-Sun Dec 16 2018 14:50:54 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:50:55 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:50:55 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:50:55 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:50:55 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:50:56

the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:17 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:17 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:17 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:17 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:17 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:18 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:18 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:18 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:18 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:18 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:19 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:51:19 GMT+0800 (Malaysia

...

the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:48 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:48 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:49 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:49 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:49 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:49 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:49 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:50 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:50 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:50 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:51 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:57:51 GMT+0800 (Malaysia

the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia

the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 14:58:41 GMT+0800 (Malaysia

the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:05 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:06 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:06 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:06 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:06 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:06 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:07 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:07 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:07 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:07 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:08 GMT+0800 (Malaysia Time)'
the vertical bangsar south cam 1-Sun Dec 16 2018 15:02:08 GMT+0800 (Malaysia

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))